In [1]:
import pandas as pd
import openpyxl
import os
import re

#合成2019-2024年的文章，输出合并的矩阵和pmids列表；
### 240821-2这个是我和林献目前分析国自然资助人群用的数据
#txt_file = "./paper_donload/paper_all_matrix.240821-2.txt" 
#pmid_file = "./paper_donload/paper_all_pmids.240821-2.txt" 

### 240922这个是5年半的所有文章的信息全部整合出来，以便于进行关键词搜索，看是否可以写综述。
txt_file = "./paper_donload/paper_all_matrix.240922.containForeign.txt" 
pmid_file = "./paper_donload/paper_all_pmids.240922.containForeign.txt"

#txt_file = "./paper_donload/paper_1wmore_matrix.txt" 
#pmid_file = "./paper_donload/paper_1wmore_pmids.txt" 

mylist = ['2024','2023','2022','2021','2020','2019','1wmore','2018','2017','2016','2015','2014','2013','2012','2011','2010']
#mylist = ['test']
#mylist = ['1wmore']

def get_text_before_paren(text):
    # 正则表达式匹配第一个括号或冒号前的字符
    match = re.search(r'(.*?)(\(|\:|\-)', text)
    if match:
        return match.group(1).strip()
    else:
        return text.strip()

def list_excel_files(directory):
    """
    遍历指定目录下所有以.xlsx结尾的文件，并返回文件路径列表。
    :return: 文件路径列表
    """
    return [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(directory)) for f in fn if f.endswith('.xlsx')]

def extract_characters_before_30th_semicolon(input_string):
    # 初始化计数器和结果字符串
    count = 0
    result = ""
    # 遍历字符串中的每个字符
    for char in input_string:
        # 如果当前字符是分号
        if char == ';':
            count += 1
            if count == 30:
                break
        # 将字符添加到结果字符串中
        result += char

    return result        

marker = 0
with open(txt_file, 'w', encoding='utf-8') as f: 
    with open(pmid_file, 'w', encoding='utf-8') as pmids:
        for i in mylist:
            directory = './paper_donload/' + i + '/'
            excel_files = list_excel_files(directory)
            for path in excel_files:
                wb = openpyxl.load_workbook(path)
                ws = wb["Sheet"]
                if marker == 0:
                    first_row = []
                    for cell in ws[1]:
                        first_row.append(str(cell.value) if cell.value is not None else '')
                    line = '\t'.join(first_row) + '\n'
                    f.write(line)
                marker = marker +1
                for row in ws.iter_rows(min_row=2,values_only=True):
                    pmids.write(str(row[0]) + '\n')
                    #if len(row) >= 3 and isinstance(row[3], str) and 'China' in row[3]:
                    if len(row) >= 3 and isinstance(row[3], str):
                        row_data = []
                        for cell in row:
                            #如果某个单元格的长度长于2000个字符，则提取第30个";"前的字符输出；
                            if isinstance(cell, str) and len(cell) > 2000:
                                cell = extract_characters_before_30th_semicolon(cell)
                                row_data.append(cell)
                            else:
                                row_data.append(cell)
                        f.write('\t'.join(str(cell) if cell is not None else '' for cell in row_data) + '\n')

